In [ ]:
from google.colab import files
files.upload()

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats as st
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV, CalibrationDisplay
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import optuna
import json
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style("darkgrid")


In [ ]:
import warnings
warnings.filterwarnings("ignore")

**Data Summary¶**
If you want to work on cleaned dataset you can directly move to data visualization section, data summary and cleaning section deals with messy and uncleaned data from which after through preprocessing Loan Credit Score Classification Problem dataset is created.

In [ ]:
train=pd.read_csv('train.csv',encoding='latin')
test=pd.read_csv('test.csv',encoding='latin')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe().T

In [ ]:
100*train.isnull().sum()

**Data Cleaning¶**
On closely observing data, the conslusion is data contains 8 consecutive months data for 12500 customers, while much information remains same for 8 months data like **Name,Annual_Income, Num_Bank_Accounts, Interest_Rate,Num_of_Loan, Outstanding_Debt**,some variables changes evry monthly **Amount_invested_monthly, Monthly_Balance** while there are some variables that dependent on previous values and increments like **Num_of_Delayed_Payment, Num_Credit_Inquiries, Credit_History_Age.**

There are also outliers and wrong information present like Payment_Behaviour=="!@9#%8", negative values for ***Num_Bank_Accounts, Num_of_Loan*** and extremely high value for Amount_invested_monthly etc.

For variables where infromation remain same throughout 8 months, we will caluate mode and replace null and wrong values with mode.

For variables that vary monthly, we will replace outliers and null values using the mode caluated for each customer_id.

For variables that increments, we will use past and future values to impute missing values using forward and backward fill.
There are many variables that should be numerical type but present as object type like **Num_of_Loan, Num_of_Delayed_Payment, Changed_Credit_Limit, Outstanding_Debt, Amount_invested_monthly**. These variables contain mixed types some instances as int/float while others as string.
**ID, Customer_ID, Month, Name, SSN, Occupation, Type_of_Loan** will be dropped as they are not useful for classification task.

Categorical variable

In [ ]:
data=train.copy()

In [ ]:
#Categorical Variables
col=["Credit_Mix", "Payment_of_Min_Amount", "Payment_Behaviour", "Type_of_Loan"]

In [ ]:
data["Credit_Mix"].value_counts()

In [ ]:
index=data[data["Credit_Mix"]=="_"].index
data.loc[index,"Credit_Mix"]=np.nan
df=data[-data["Credit_Mix"].isna()]
ser=df.groupby("Customer_ID")["Credit_Mix"].agg(st.mode).apply(lambda x:x[0][0])
data.drop("Credit_Mix",axis=1,inplace=True)
data=pd.merge(left=data,right=ser,how="left",left_on="Customer_ID",right_index=True)

In [ ]:
data["Payment_Behaviour"].value_counts()

In [ ]:
index=data[data["Payment_Behaviour"]=="!@9#%8"].index
data.loc[index,"Payment_Behaviour"]=np.nan
df=data[-data["Payment_Behaviour"].isna()]
ser=data.groupby("Customer_ID")["Payment_Behaviour"].agg(st.mode).apply(lambda x:x[0][0])
data.drop("Payment_Behaviour",axis=1,inplace=True)
data=pd.merge(left=data,right=ser,how="left",left_on="Customer_ID",right_index=True)

In [ ]:
data["Payment_of_Min_Amount"].value_counts()

**Numerical Variable**

In [ ]:
'''Features that should be of numerical type but are present as categorical
   variable(object type), we will convert them to numerical type.These variable
   contain mix type, for example Monthly_Balance variable contain float type for
   index 0:98303 and string type from index 98304:99999'''

cols=['Age','Annual_Income','Num_of_Loan','Num_of_Delayed_Payment','Changed_Credit_Limit',
      'Outstanding_Debt','Amount_invested_monthly','Monthly_Balance']

for col in cols:
    data[col]=data[col].apply(lambda x: x if x is np.nan or not isinstance(x, str)
                        else x.replace("_","")).replace("",np.nan)
    data[col]=data[col].astype(np.float64)

In [ ]:
cols=["Age", "Annual_Income", "Num_Bank_Accounts","Num_Credit_Card", "Interest_Rate",
      "Num_of_Loan", "Monthly_Inhand_Salary","Changed_Credit_Limit","Outstanding_Debt",
      "Total_EMI_per_month"]

for col in cols:
    index=data[data[col]<0].index
    data.loc[index,col]=np.nan
    df=data[-data[col].isna()]
    ser=df.groupby("Customer_ID")[col].agg(st.mode).apply(lambda x:x[0][0])
    data.drop(col,axis=1,inplace=True)
    data=pd.merge(left=data,right=ser,how="left",left_on="Customer_ID",right_index=True)

In [ ]:
'''These features can best imputed by same feature values present in customer record for
   different month,therefore we will use that customer record for imputating missing
   value using ffill and bfill'''

inter=data["Credit_History_Age"].str.split(" ")
data["Credit_History_Age"]=inter.apply(lambda x:x if x is np.nan else int(x[0])*12+int(x[3]))

cols=['Num_of_Delayed_Payment', 'Num_Credit_Inquiries',"Credit_History_Age",
      "Delay_from_due_date"]
for col in cols:
    if col in ['Num_of_Delayed_Payment', 'Num_Credit_Inquiries']:
        index2=data[(data[col]>=30)].index
        data.loc[index2,col]=np.nan
    index1=data[(data[col]<0)].index
    data.loc[index1,col]=np.nan
    data[col]=data.groupby("Customer_ID")[col].transform(lambda x:x.ffill())
    data[col]=data.groupby("Customer_ID")[col].transform(lambda x:x.bfill())

In [ ]:
for col in ['Amount_invested_monthly', 'Monthly_Balance']:
    if col in ['Amount_invested_monthly']:
        index2=data[(data[col]==10000.0)].index
        data.loc[index2,col]=np.nan
    index1=data[(data[col]<0)].index
    data.loc[index1,col]=np.nan
    ser=data.groupby("Customer_ID")[col].transform(lambda x:x.median())
    data[col].fillna(ser,inplace=True)

In [ ]:
data.describe().T

In [ ]:
#Redundant variable will be dropped

drop=["ID", "Customer_ID", "Month","Name", "SSN", "Occupation","Type_of_Loan"]
data.drop(columns=drop,axis=1,inplace=True)

In [ ]:
data.dropna(axis=0, inplace=True)

In [ ]:
data.to_csv("Score.csv",index=False)

**Data Visualization**

In [ ]:
data=pd.read_csv("/kaggle/input/processed-data-credit-score/Score.csv")

**Clean Data Features Description**
**Age:** Represents the age of the person

**Annual_Income:** Represents the annual income of the person

**Monthly_Inhand_Salary:** Represents the monthly base salary of a person

**Num_Bank_Accounts:**Represents the number of bank accounts a person holds

**Num_Credit_Card:** Represents the number of other credit cards held by a person

**Interest_Rate:** Represents the interest rate on credit card

**Num_of_Loan:** Represents the number of loans taken from the bank

**Delay_from_due_date:** Represents the average number of days delayed from the payment date

**Num_of_Delayed_Payment:** Represents the average number of payments delayed by a person

**Changed_Credit_Limit:** Represents the percentage change in credit card limit

**Num_Credit_Inquiries:** Represents the number of credit card inquiries

**Credit_Mix:** Represents the classification of the mix of credits

**Outstanding_Debt:** Represents the remaining debt to be paid (in USD)

**Credit_Utilization_Ratio:** Represents the utilization ratio of credit card

**Credit_History_Age:** Represents the age of credit history of the person

**Payment_of_Min_Amount:** Represents whether only the minimum amount was paid by the person

**Total_EMI_per_month:** Represents the monthly EMI payments (in USD)

**Amount_invested_monthly:** Represents the monthly amount invested by the customer (in USD)

**Monthly_Balance:** Represents the monthly balance amount of the customer (in USD)

**Credit_Score:** Represents the bracket of credit score (Poor, Standard, Good)

In [ ]:
X=data.drop("Credit_Score",axis=1)
y=data["Credit_Score"]

numeric=X.select_dtypes(exclude="object").columns
categorical=['Payment_of_Min_Amount', 'Credit_Mix']

low_cardinality=[col for col in numeric if data[col].nunique()<=30]
high_cardinality=[col for col in numeric if data[col].nunique()>30]

In [ ]:
ser=data["Credit_Score"].value_counts()
plt.pie(x=ser.values, labels=ser.index, autopct="%0.2f%%", radius=2);
plt.savefig("image1.png")

**There is unequal class distribution in target variable,but the difference ratio is not very high. we will test cost-sensitive learning on the dataset.**

In [ ]:
rows=[len(high_cardinality)//2 if len(high_cardinality)%2==0 else len(high_cardinality)//2+1]
fig,axes=plt.subplots(rows[0],2,figsize=(10,30))

ax=axes.flatten()
for i,col in enumerate(high_cardinality):
    sns.histplot(data=data,x=col,hue="Credit_Score",ax=ax[i],multiple="stack");

plt.tight_layout()
plt.savefig("image2.png")

**Outstanding_Debt,Interest_Rate,Delay_from_due_date** as these variables value increases, the proportion of Bad Credit_Score instances increases substantially, while Standard and Good Credit_Score instances proportion become meagre.

On the other hand increase in **Credit_History_Age, Age** reults in decrease of Bad Credit_Score instances, jump in Standard and Good Credit_Score instance.
Interestingly credit_utilization_ratio seems to seems to have same distribution for all categories of credit score.

The distribution of many features have non-gaussian distribution and largely different scales, therefore we will tranform feature variables.

In [ ]:
cols=["Outstanding_Debt", "Interest_Rate", "Delay_from_due_date", "Credit_History_Age"]
vals=["Good","Standard","Poor"]
colors=["blue","orange","green"]

fig,axes=plt.subplots(len(cols),3,figsize=(15,25))

for i,col in enumerate(cols):
    for j,(val,color) in enumerate(zip(vals,colors)):
        ax=axes[i,j]
        inter=data[data["Credit_Score"]==val]
        sns.histplot(data=inter,x=col,ax=ax,color=color,kde=True);
        ax.set_title(f"{val} distribution for {col}")

plt.tight_layout()
plt.savefig("image3.png")


In [ ]:
plt.figure(figsize=(8, 6))
arr=np.ones_like(data[high_cardinality].corr())
mask=np.triu(arr)
sns.heatmap(data[high_cardinality].corr(), cbar=False, annot=True, fmt=".2g", mask=mask);
plt.savefig("image4.png")


Many variable have strong correlation with others,for example **Num_of_Delayed_Payment** has strong positive correlation with interest_rate charged(0.57), outstanding_debt(0.50), delay_from_due_date (0.54), Num_Credit_Inquiries (0.50) and strong negative correlation with Credit_History_Age (-0.49) and Monthly_Inhand_Salary(-0.28).

**A person who has taken large sum of money at high interest rate with less salary, financial discipline and savings are more likely to have high no of delayed payments, looking for more credit to pay off loans, ending up with high interest rate and large debt.**

**Credit_History_Age** has strong negative correlation with Num_of_Delayed_Payment, Num_Credit_Inquiries, Interest_Rate, Outstanding_Debt.

In [ ]:
fig,axes=plt.subplots(2,2,figsize=(10,12))

for i,col in enumerate(categorical[:-1]):
    ax=axes[i,0]
    sns.countplot(x=data[col],hue=data["Credit_Score"],ax=ax)
    ax.set_xticklabels(labels=data[col].unique(),rotation=90)
    ax.legend()

    ax=axes[i,1]
    sns.violinplot(x=data[col],hue=data["Credit_Score"],y=data["Outstanding_Debt"],ax=ax)
    ax.set_xticklabels(labels=data[col].unique(),rotation=90)
    ax.legend()

plt.tight_layout()
plt.savefig("image5.png")

We can see **Credit_Mix** and **Payment_of_Min_Amount** influences the categories of Credit_Score

If someone has made Payment_of_Min_Amount, poor score have higher outstanding debt than standard and good debt, while if someone has not made Payment_of_Min_Amount then outstanding debt have no impact on credit score.

In [ ]:
fig,axes=plt.subplots(5,figsize=(10,20))

for i,col in enumerate(low_cardinality):

    ax=axes[i]
    sns.violinplot(x=data[col],y=data["Credit_Score"],ax=ax);
    ax.set_xticklabels(labels=data[col].unique(),rotation=90)

plt.tight_layout()
plt.savefig("image6.png")


As the variables **Num_of_Delayed_Payment**,** Num_Credit_Inquiries**, **Num_Bank_Accounts, Num_Credit_Card, Num_of_Loan** values increases the proportion of poor credit scores increases, while good and standard credit scores instances were quite less.

**Num_of_Delayed_Payment** variable having the most visible effect.

In [ ]:
from itertools import product
arr1=["Credit_History_Age","Monthly_Inhand_Salary"]
arr2=["Outstanding_Debt","Interest_Rate","Delay_from_due_date"]

pairs=list(product(arr1,arr2))

fig,axes=plt.subplots(len(pairs),3,figsize=(15,30))

for i,(x,y) in enumerate(pairs):
    colors=["blue","orange","green"]
    vals=["Good","Standard","Poor"]
    for j,(color,val) in enumerate(zip(colors,vals)):
        ax=axes[i,j]
        inter=data[data["Credit_Score"]==val]
        sns.histplot(x=x,y=y,data=inter,color=color,ax=ax);

plt.tight_layout()
plt.savefig("image7.png")

Theses charts shows density distribution of credit scores categories using bivariate analysis.

**Overlapping instances zones are places where instances are mostly misclassified, seeing the data ensemble based learning methods will be best, while linear models will be ineffective because of presence of complex non-linear relationship.SVM will be time consuming because of large dataset size, overlapping of instances.**

**Data Preprocessing**

In [ ]:
#label encoding target variable
label=LabelEncoder()
label.fit(data["Credit_Score"])
y=label.transform(data["Credit_Score"])

#transforming and scaling numerical variable
transformer=PowerTransformer()
transformer.fit(data[high_cardinality])
numeric=transformer.transform(data[high_cardinality])

#one hot encoding
encoding=OneHotEncoder(drop="first")
encoding.fit(data[categorical])
one_hot=encoding.transform(data[categorical]).toarray()

#ordinal variables
ordinal=data[low_cardinality].values

#Feature array
X=np.concatenate([numeric,one_hot,ordinal],axis=1)

**Algorithm Spot Checking**

In [ ]:
def get_models():
    names,models=[],[]

    models.append(RandomForestClassifier(random_state=42,n_jobs=-1))
    names.append("RandomForestClassifier")

    models.append(XGBClassifier(random_state=42,n_jobs=-1))
    names.append("XGBClassifier")

    models.append(LGBMClassifier(random_state=42,n_jobs=-1))
    names.append("LGBMClassifier")

    return names,models

In [ ]:
names,models=get_models()
fig,axes=plt.subplots(3,2,figsize=(12,15))
result={}

for i in range(len(models)):
    scores=[]
    importance=[]
    probability=[]

    for train_idx,val_idx in StratifiedKFold(n_splits=5).split(X,y):
        model=models[i].fit(X[train_idx,:],y[train_idx])
        y_pred=model.predict(X[val_idx,:])
        y_true=y[val_idx]
        score=f1_score(y_true,y_pred,average='macro')

        scores.append(score)
        importance.append(model.feature_importances_)
        probability.append(model.predict_proba(X[val_idx,:]))

    ax=axes[i,0]
    frame=pd.DataFrame(importance)
    result_p=pd.melt(frame,value_vars=frame.columns)
    sns.barplot(x="variable",y="value",data=result_p,ax=ax);
    ax.set_title(f"{names[i]} model feature importance")

    ax=axes[i,1]
    y_true=label_binarize(y,classes=range(len(label.classes_)))
    y_proba=np.concatenate([probability[j] for j in range(len(probability))])
    for k in range(len(label.classes_)):
        precision,recall,thresh=precision_recall_curve(y_true[:,k],y_proba[:,k])
        sns.scatterplot(recall, precision, alpha=0.95,ax=ax,label=label.classes_[k])
    ax.legend()
    ax.set_title(f"{names[i]} model precision recall curve")

    plt.tight_layout()
    result[names[i]]=scores

plt.savefig("image8.png")

While the precision recall curve is nearly same for all three classifier, different features are hold diffrenet significance for models, with xgboost model feature importance quite from other model. This signifies that model are diffrent and using their values in an ensemble or stacking might increase performance. Also lgbm model performed better than xgboost and random forest classifier.

The precision recall curve also tells us that good credit score is misclssified more than standard and poor credit score class. Standard Credit Score class is misclassified least across all three models.

The models are performing better on standard instances, underperforming on good and poor instances this can be due to class imbalance. Therefore we can use data sampling technique specially those that increase the instances of classes in borderline or overlapping area.

In [ ]:
plt.figure(figsize=(10,8))
frame=pd.DataFrame(result)
result_p=pd.melt(frame,value_vars=frame.columns)
sns.violinplot(x="variable",y="value",data=result_p);
plt.savefig("image9.png")

**LGBMClassifier**

In [ ]:
def objective(trial):

    params={"n_estimators":trial.suggest_int("n_estimators",100,400),
            "max_depth":trial.suggest_int("max_depth",3,8),
            "learning_rate":trial.suggest_float("learning_rate", 0.01, 0.1,log=True),
            "subsample":trial.suggest_float("subsample",0.2,0.8),
            "colsample_bytree": trial.suggest_float("colsample_bytree",0.2,0.8),
            "reg_lambda":trial.suggest_float("reg_lambda", 0.01, 0.1,log=True)}

    model=LGBMClassifier(**params,random_state=42,n_jobs=-1)
    cv=StratifiedKFold(n_splits=5)
    scores=cross_val_score(model,X,y,scoring="f1_macro",cv=cv,n_jobs=-1)
    trial.set_user_attr("f1_macro", scores.mean())

    return scores.mean()

In [ ]:
study = optuna.create_study(study_name="Hyperparameter optimization",direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=20,show_progress_bar=True)

In [ ]:
print(f"Best value: {study.best_trial.value}")
print(f"Best hyperparameters:\n {json.dumps(study.best_trial.params, indent=2)}")

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)
plt.savefig("image10.png")

In [ ]:
params=["n_estimators","max_depth","learning_rate","colsample_bytree"]
plot_parallel_coordinate(study,params=params)
plt.savefig("image11.png")

**XGBClassifier**

In [ ]:
def objective(trial):

    params={"n_estimators":trial.suggest_int("n_estimators",100,400),
            "max_depth":trial.suggest_int("max_depth",3,8),
            "learning_rate":trial.suggest_float("learning_rate", 0.01, 0.1,log=True),
            "subsample":trial.suggest_float("subsample",0.2,0.8),
            "colsample_bytree": trial.suggest_float("colsample_bytree",0.2,0.8),
            "reg_lambda":trial.suggest_float("reg_lambda", 0.01, 0.1,log=True)}

    model=XGBClassifier(**params,random_state=42,n_jobs=-1)
    cv=StratifiedKFold(n_splits=5)
    scores=cross_val_score(model,X,y,scoring="f1_macro",cv=cv,n_jobs=-1)
    trial.set_user_attr("f1_macro", scores.mean())

    return scores.mean()

In [ ]:
study = optuna.create_study(study_name="Hyperparameter optimization",direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=20,show_progress_bar=True)

In [ ]:
print(f"Best value: {study.best_trial.value}")
print(f"Best hyperparameters:\n {json.dumps(study.best_trial.params, indent=2)}")

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)
plt.savefig("image12.png")

In [ ]:
params=["n_estimators","max_depth","colsample_bytree"]
plot_parallel_coordinate(study,params=params)
plt.savefig("image13.png")

**RandomForestClassifer**

In [ ]:
def objective(trial):

    params={"n_estimators":trial.suggest_int("n_estimators",100,400),
            "max_depth":trial.suggest_int("max_depth",3,8),
            "max_samples":trial.suggest_float("max_samples",0.2,0.8),
            "min_samples_split": trial.suggest_int("min_samples_split",2,20),
            "max_features":trial.suggest_categorical("max_features", ["auto", "sqrt", "log2"])}

    model=RandomForestClassifier(**params,random_state=42,n_jobs=-1)
    cv=StratifiedKFold(n_splits=5)
    scores=cross_val_score(model,X,y,scoring="f1_macro",cv=cv,n_jobs=-1)
    trial.set_user_attr("f1_macro", scores.mean())

    return scores.mean()

In [ ]:
study = optuna.create_study(study_name="Hyperparameter optimization",direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=20,show_progress_bar=True)

In [ ]:
print(f"Best value: {study.best_trial.value}")
print(f"Best hyperparameters:\n {json.dumps(study.best_trial.params, indent=2)}")

In [ ]:
plot_optimization_history(study)

In [ ]:
plot_param_importances(study)
plt.savefig("image14.png")

In [ ]:
params=["max_depth", "max_samples", "min_samples_split"]
plot_parallel_coordinate(study,params=params)
plt.savefig("image15.png")

**Ensemble of Models**

In [ ]:
lgbm_params={"n_estimators": 319,
             "max_depth": 5,
             "learning_rate": 0.01007506187129231,
             "subsample": 0.7243746196920224,
             "colsample_bytree": 0.6043128820104816,
             "reg_lambda": 0.010068213652019799 }

xgb_params={"n_estimators": 319,
            "max_depth": 5,
            "learning_rate": 0.010320226911021985,
            "subsample": 0.41644521272647506,
            "colsample_bytree": 0.6465159042623695,
            "reg_lambda": 0.04479410490748427 }

rf_params={"n_estimators": 212,
           "max_depth": 8,
           "max_samples": 0.6391963650868431,
           "min_samples_split": 13,
           "max_features": "auto"}

In [ ]:
models={"LGBMClassifier":LGBMClassifier(**lgbm_params,random_state=42,n_jobs=-1),
        "XGBClassifier":XGBClassifier(**xgb_params,random_state=42,n_jobs=-1),
        "RandomForestClassifier":RandomForestClassifier(**rf_params,random_state=42,n_jobs=-1)}

In [ ]:
probability=dict(zip(models.keys(),[[] for _ in range(len(models.keys()))]))

for train_idx,val_idx in StratifiedKFold(n_splits=5).split(X,y):
    X_train,y_train=X[train_idx,:],y[train_idx]
    X_val,y_val=X[val_idx,:],y[val_idx]
    for name,model in models.items():
        algo=model.fit(X_train,y_train)
        y_proba=algo.predict_proba(X[val_idx,:])
        probability[name].append(algo.predict_proba(X[val_idx,:]))

In [ ]:
inter=probability.copy()

In [ ]:
y_true=label_binarize(y,classes=range(len(label.classes_)))
probas=dict(zip(models.keys(),[[] for _ in range(len(models.keys()))]))
for name in models.keys():
    proba=np.concatenate([probability[name][i] for i in range(len(probability[name]))])
    probas[name].append(proba)

In [ ]:
fig,axes=plt.subplots(3,1,figsize=(15,15))
ax=axes.flatten()

for i,classes in enumerate(list(label.classes_)):
    ax[i].set_title(f"Calibration Curve for {classes}",fontweight="bold")
    for name,prediction in probas.items():
        y_real=y_true[:,i]
        y_prob=probas[name][0][:,i]
        CalibrationDisplay.from_predictions(y_real,y_prob,n_bins=10, strategy='quantile',ax=ax[i],name=name);

plt.tight_layout()
plt.savefig("image16.png")

**The probabilities predicted by the models for each class in target variable is well calibrated. Therefore there is no need to calibrate the predicted probabailities.**

In [ ]:
probas["Average_Models"]=[np.add(probas["LGBMClassifier"][0],probas["XGBClassifier"][0])]
probas["Average_Models"]=[np.add(probas["Average_Models"][0],probas["RandomForestClassifier"][0])/3]

In [ ]:
plt.figure(figsize=(15,15))
axes=plt.subplot(1,1,1)

for i,classes in enumerate(list(label.classes_)):
    y_real=y_true[:,i]
    y_prob=probas["Average_Models"][0][:,i]
    CalibrationDisplay.from_predictions(y_real,y_prob,n_bins=10, strategy='quantile',name=classes,ax=axes);
plt.savefig("image17.png")

In [ ]:
for models in probas.keys():
    sol=np.argmax(probas[models][0],axis=1)
    score=f1_score(y,sol,average="macro")
    print(f"\n{models} f1 score is: {score}")

**The ensemble of model performed better than individual model, we can also try weights ensemble and stacking. Plus data oversampling of minority instances classes and undersampling of overlapping instances using imblearn library.**

In [ ]:
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix(y,sol),annot=True,cbar=False,xticklabels=label.classes_,yticklabels=label.classes_,cmap="Blues");
plt.savefig("image18.png")

In [ ]:
print(classification_report(y,sol))